In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.atlassian.com/company/careers/details/16968")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.



         Job Details | Atlassian                                  Close   View this page in your language?   All languages   Choose your language    中文   Deutsch   English   Español   Français   Italiano   한국어   Nederlands   日本語   Português   Pусский   Polski                            Try now     Products    Featured    Developers    Product Managers    IT professionals    Business Teams    Leadership Teams       Featured   Developers   Product Managers   IT professionals   Business Teams   Leadership Teams    See all products       Featured            Jira  Flexible project management             Confluence  Knowledge, all in one place             Jira Service Management  High-velocity service delivery             Trello  Organized & visualized work             Rovo NEW  Unlock enterprise knowledge             Jira Product Discovery NEW  Capture & prioritize ideas             Compass NEW  Optimize software health             Guard NEW  Enhanced cloud security             Loom NEW  Q

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

# Ensure the output is a dictionary
if isinstance(json_res, list) and len(json_res) > 0:
    json_res = json_res[0] 

print(type(json_res)) 
print(json_res) 

<class 'dict'>
{'role': 'Site Reliability Engineer', 'experience': ['1+ years experience operating high-availability, fault-tolerant, scalable, distributed software in production', '1+ years of hands-on experience with public cloud offerings (AWS components like EC2, CloudFormation, RDS / Aurora, Caches, SQS - or equivalents, e.g. in GCP / Azure)'], 'skills': ['Familiarity with Unix / Linux operating systems', 'Backend engineering experience in one or more prominent languages such as Java, Go or Python', 'Strong communication skills in written and verbal forms', 'Experience implementing caching solutions, strategies, and best practices', 'Experience in microservice architecture', 'Experience building web-services and clients using REST/GraphQL'], 'description': "We are looking for an engineer who is passionate about scaling cloud services to join our growing SRE team. The SRE team owns the caching infrastructure, tooling, and automation that support Atlassian’s suite of Cloud products.

In [9]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"AWS, Azure, Python, Go",https://example.com/aws-portfolio
1,"React, Node.js, MongoDB",https://example.com/react-portfolio
2,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
3,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
4,"Python, Django, MySQL",https://example.com/python-portfolio
5,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
6,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
7,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
8,"Magento, PHP, MySQL",https://example.com/magento-portfolio
9,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [17]:
job = json_res
job['skills']

['Familiarity with Unix / Linux operating systems',
 'Backend engineering experience in one or more prominent languages such as Java, Go or Python',
 'Strong communication skills in written and verbal forms',
 'Experience implementing caching solutions, strategies, and best practices',
 'Experience in microservice architecture',
 'Experience building web-services and clients using REST/GraphQL']

In [18]:
links = collection.query(query_texts=["Experience in AWS"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/aws-portfolio'},
  {'links': 'https://example.com/cloud-portfolio'}]]

In [20]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Lokesh, a Solutions Specialist at XYZ company. XYZ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XYZ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase XYZ's portfolio: {link_list}
        Remember you are Lokesh, Solutions Specialist at XYZ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)


chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Scaling Cloud Services and Enhancing Reliability

Dear Hiring Manager,

I came across the Site Reliability Engineer role at Atlassian and was impressed by the company's commitment to delivering high-quality cloud services. As a Solutions Specialist at XYZ, I'd like to introduce our company's expertise in facilitating seamless integration of business processes through automated tools.

Our team at XYZ has extensive experience in empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. We believe our capabilities align perfectly with Atlassian's requirements for scaling cloud services and enhancing reliability.

Our expertise in cloud infrastructure, programming, and operational experience make us an ideal partner for Atlassian's SRE team. We have hands-on experience with public cloud offerings, including AWS components like EC2, CloudFormation, RDS/Aurora, Caches, and SQS